<center><font size = "10"> Week 12 - Network Simulation III<center>
<center><font size = "8">Tutorial 03: Detailed morphologies <center>

<font size='5'><font color='red'>__IMPORTANT!!__ Read the document "ReadBefore.pdf" before running this tutorial or any of the week 12 tutorials

<font size='3'><font color='blue'>In The following tutorial we will see how to use detailed morphological reconstructed neurons and build a network using __NetPyNe__.

## Step 1: Import
<font size='3'>Import the __sim__ and __specs__ modules from the __netpyne__ package, that provide functions to easily create and simulate networks.

In [ ]:
from netpyne import specs, sim

%matplotlib inline

## Step 2: Define network parameters

In [ ]:
# Network parameters
netParams = specs.NetParams()  # object of class NetParams to store the network parameters

### 2.1 Define cell populations

In [ ]:
## Population parameters
netParams.popParams['E1'] = {'cellType': 'E', 'numCells': 5, 'cellModel': 'HH3D'}
netParams.popParams['I1'] = {'cellType': 'I', 'numCells': 5, 'cellModel': 'HH3D'}

### 2.2 Define cell morphologies and physiology (as .py or .hoc files)

In [ ]:
cellRule = netParams.importCellParams(label='PYR1_HH3D', conds={'cellType': 'E', 'cellModel': 'HH3D'},
        fileName='Pyr01.py', cellName='PYR1', importSynMechs=True)
netParams.cellParams['Erule'] = cellRule  

cellRule = netParams.importCellParams(label='INH1_HH3D', conds={'cellType': 'I', 'cellModel': 'HH3D'},
        fileName='Inh01.py', cellName='INH1', importSynMechs=True)
netParams.cellParams['Irule'] = cellRule 

### 2.3 Define synaptic mechanism

In [ ]:
## Synaptic mechanism parameters
netParams.synMechParams['exc'] = {'mod': 'Exp2Syn', 'tau1': 0.8, 'tau2': 5.3, 'e': 0}  # NMDA synaptic mechanism
netParams.synMechParams['inh'] = {'mod': 'Exp2Syn', 'tau1': 0.6, 'tau2': 8.5, 'e': -75}  # GABA synaptic mechanism

### 2.4 Stimulation

In [ ]:
# Stimulation parameters
netParams.stimSourceParams['bkg'] = {'type': 'NetStim', 'rate': 20, 'noise': 0.3}
netParams.stimTargetParams['bkg->all'] = {'source': 'bkg', 'conds': {'cellType': ['E','I']}, 'weight': 0.1, 'delay': 'max(1, normal(5,2))', 'synMech': 'exc'}

### 2.5 Connectivity

In [ ]:
## Cell connectivity rules
netParams.connParams['I->E'] = { #  I1 -> E1 label
    'preConds': {'pop': 'I1'},   # conditions of presyn cells
    'postConds': {'pop': 'E1'},  # conditions of postsyn cells
    'divergence': 12,            # probability of connection
    'weight': 0.5,               # synaptic weight
    'delay': 5,                  # transmission delay (ms)
    'synMech': 'inh'}            # synaptic mechanism   

## Step 3: Simulation parameters

In [ ]:
# Simulation options
simConfig = specs.SimConfig()           # object of class SimConfig to store simulation configuration

simConfig.duration = 0.5*1e3            # Duration of the simulation, in ms
simConfig.dt = 0.025                    # Internal integration timestep to use
simConfig.verbose = False               # Show detailed messages
simConfig.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}  # Dict with traces to record
simConfig.recordStep = 0.1              # Step size in ms to save data (eg. V traces, LFP, etc)
simConfig.filename = 'model_output'     # Set file output name
simConfig.savePickle = False            # Save params, network and sim output to pickle file
simConfig.saveJson = True 

# Plots
simConfig.analysis['plotRaster'] = True       # Plot a raster
simConfig.analysis['plot2Dnet'] = True        # plot 2D visualization of cell positions and connections

### Run simulation

In [ ]:
# Create network and run simulation
sim.createSimulateAnalyze(netParams = netParams, simConfig = simConfig)